# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# Load the csv to a dataframe
city_data = pd.read_csv("../output_data/cities.csv")

# Display the dataframe
city_data

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,Albany County,75,US,1589836347,63,42.6001,-73.9662,64.40,8.05
1,1,Hithadhoo,100,MV,1589835969,75,-0.6000,73.0833,84.60,23.02
2,2,Khovd,33,MN,1589836176,37,48.0056,91.6419,57.52,2.04
3,3,Panlaitan,59,PH,1589836281,72,12.1191,119.8486,84.42,1.52
4,4,Mataura,11,NZ,1589836648,89,-46.1927,168.8643,36.10,2.10
...,...,...,...,...,...,...,...,...,...,...
543,543,Pacific Grove,75,US,1589836622,49,36.6177,-121.9166,68.00,11.41
544,544,Tuzha,100,RU,1589835917,85,57.6062,47.9360,46.22,6.35
545,545,Ciudad Lázaro Cárdenas,65,MX,1589836570,45,17.9583,-102.2000,85.91,12.59
546,546,Atambua,100,ID,1589836179,79,-9.1061,124.8925,73.49,5.88


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps
gmaps.configure(api_key=g_key)

In [4]:
# Store latitude and longitude in locations
locations = city_data[["Lat", "Lng"]]
humidity = city_data["Humidity"].astype(float)

# Plot Heatmap
fig = gmaps.figure(zoom_level = 2, center = (34.0522,118.2437))

# Create heat layer
heat_layer = gmaps.heatmap_layer(
    locations, 
    weights=humidity, 
    dissipating=False, 
    max_intensity=humidity.max(),
    point_radius=3)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
# Narrow down the cities to find the ideal cities with maximum temperature between 70-80 degree, wind speed less than 10 mph, and zero cloudiness
ideal_weather = city_data.loc[
    (city_data["Max Temp"]<80) & (city_data["Max Temp"]>70) & (city_data["Wind Speed"]<10) & (city_data["Cloudiness"]==0),:
]

# Display the dataframe
ideal_weather 

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
117,117,Trairi,0,BR,1589836522,81,-3.2778,-39.2689,79.68,9.28
158,158,Namuac,0,PH,1589835983,78,18.6109,121.1657,78.30,4.34
181,181,Morón,0,AR,1589836527,61,-34.6534,-58.6198,71.01,7.70
190,190,Tura,0,IN,1589836162,42,25.5198,90.2201,77.90,2.84
194,194,Elvas,0,PT,1589836350,47,38.8815,-7.1628,75.20,2.24
211,211,Hami,0,CN,1589836159,17,42.8000,93.4500,72.64,8.19
230,230,Yulin,0,CN,1589836163,86,22.6333,110.1500,73.94,0.51
340,340,Fuerte Olimpo,0,PY,1589836244,50,-21.0415,-57.8738,77.04,4.47
514,514,General Pico,0,AR,1589836561,44,-35.6566,-63.7568,70.75,8.50


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
# Create a new dataframe for gmaps, narrow down the data we need
hotel_df = ideal_weather.loc[:,["City", "Country", "Lat", "Lng"]]

# Create a "Hotel Name" column in the dataframe
hotel_df["Hotel Name"] = ""

# Display the dataframe
hotel_df

,City,Country,Lat,Lng,Hotel Name
117,Trairi,BR,-3.2778,-39.2689,
158,Namuac,PH,18.6109,121.1657,
181,Morón,AR,-34.6534,-58.6198,
190,Tura,IN,25.5198,90.2201,
194,Elvas,PT,38.8815,-7.1628,
211,Hami,CN,42.8000,93.4500,
230,Yulin,CN,22.6333,110.1500,
340,Fuerte Olimpo,PY,-21.0415,-57.8738,
514,General Pico,AR,-35.6566,-63.7568,


In [7]:
# Specify the url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?"

# Set parameters 
params = {
    "radius": 5000,
    "type": "hotel",
    "key": g_key,
    "keyword": "hotel"
}

# Use the latitude and longitude to identify hotels
for index, row in hotel_df.iterrows():
    
    # Get latitude and longitude
    lat = row["Lat"]
    lng = row["Lng"]

    # Set the location parameter
    params["location"] = f"{lat},{lng}"
    
    # Get request from google and store it
    response = requests.get(base_url, params = params).json()

    #  Since some data may be missing the hotel name, a try-except to skip any that are missing a data point
    try:
        hotel_df.loc[index, "Hotel name"] = response["results"][0]["name"]
    
    except:
        pass

# Display the dataframe
hotel_df

,City,Country,Lat,Lng,Hotel Name,Hotel name
117,Trairi,BR,-3.2778,-39.2689,,Zorah Beach Hotel
158,Namuac,PH,18.6109,121.1657,,NaN
181,Morón,AR,-34.6534,-58.6198,,Hotel Cabildo
190,Tura,IN,25.5198,90.2201,,Hotel Polo Orchid
194,Elvas,PT,38.8815,-7.1628,,Dom Luis Hotel
211,Hami,CN,42.8000,93.4500,,哈密环球大酒店
230,Yulin,CN,22.6333,110.1500,,zhongding hotel
340,Fuerte Olimpo,PY,-21.0415,-57.8738,,Hotel AA
514,General Pico,AR,-35.6566,-63.7568,,Hotel Aires de la Pampa


In [8]:
hotel_modify_df = hotel_df[["Hotel name", "City", "Country", "Lat", "Lng"]]


In [9]:
# NOTE: Do not change any of the code in this cell
# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_modify_df.iterrows()]
locations = hotel_modify_df[["Lat", "Lng"]]


In [11]:
# Create marker layer
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)

# Add marker layer ontop of heat map
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))